In [1]:
import pandas as pd
from xgboost import XGBRegressor as xgbr
from xgboost import XGBClassifier as xgbc
import numpy as np
import random
import json

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, f1_score, roc_auc_score, log_loss

from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

np.random.seed(67)
random.seed(67)


In [2]:
df = pd.read_csv('./data/epcg23.csv')
df.shape

(94606, 548)

---
## Y label: 
**Probability of recent bachelor graduates getting a paid job within there field after graduation.**
1) **WRKG** Working for pay or profit during reference week
2) **OCEDRLP** Extent that principal job is related to highest degree
3) **DGRDG** Highest degree type
4) **STRTYR** Year principal job started
5) **DGRYR** Year of award of highest degree

```
* If 1, 2 and 3 is check out, and year of 4 is => year of 5 then Y=1
```

### Other relevant features  
* **LFSTAT** Labor force status
* **DGRDG** Highest degree type
* **HDMN** Month of award of highest degree
* **NDGMEMG** Field of study for highest degree (major group)
* **NDGMENG** Field of study for highest degree (minor group)
* **HDPBP21C** Public/private status of school awarding highest degree - 2021 Carnegie code
* **HDRGN** Location of school awarding highest degree (region code)
* **BAYR** Year of award of first bachelors degree
* **CLICWKR** Certificates and licenses: for work-related reasons
* **CLICNOW** Certification or licenses: for principal job
* **CLICISS** Certification or licenses: issuer
* **CLICCODE** Certification/license primary subject or field of study
* **CLICYR** Certification or licenses: year first issued
### **Demograhic Related**
* **AGE** Age
* **SEX_2023** Sex at birth
* **CTZN** Citizenship or visa status
* **CTZFOR** Visa type for non-US citizens
* **FNUSYR6** The year first came to U.S. for 6 months or longer
* **VETSTAT** Veteran status: served on active duty in the US Armed Forces, Reserves, or National Guard
### **Geogrophy Related**
* **RESPLOC** Respondent location
* **RESPLO3_TOGA** 3-Digit Respondent Location (state/country code)
* **RESPLCUS** Respondent location (U.S./Non-U.S.)
* **EMRG** Region code for employer
* **EMST_TOGA** State/country code for employer
### **Finacial Related**
* **UGLOANR** Amount borrowed to finance UNDERGRADUATE degree(s)
* **UGOWER** Amount still owed from financing of UNDERGRADUATE degree(s)
* **GRFLN** Financial support for graduate degree(s): Loans from school, banks, and government
* **SALARY** Salary

---
## Data Cleaning

In [3]:
y_variables = ['DGRDG','WRKG','SALARY','OCEDRLP','DGRYR','STRTYR','STRTMN','HDMN']
df[y_variables].dtypes

DGRDG       int64
WRKG       object
SALARY      int64
OCEDRLP    object
DGRYR       int64
STRTYR      int64
STRTMN      int64
HDMN        int64
dtype: object

In [4]:
# make y label

# DGRDG == 1; highest degree is bachelor
# WRKG == 'Y'; working 
# SALARY >= 1; and getting paid i.e. no internship
# OCEDRLP in {1,2}; works in field
# (DGRYR - STRTYR) < 1; job started within a year after graduation

months = (df['STRTYR'] - df['DGRYR']) * 12 + (df['STRTMN'] - df['HDMN'])

df['y'] = (
    (df['DGRDG'] == 1) &
    (df['WRKG'] == 'Y') &
    (df['SALARY'] >= 1) & (df['SALARY'] < 9999998) &
    (pd.to_numeric(df['OCEDRLP'], errors='coerce').isin([1, 2])) &
    (months.between(0, 12, inclusive='both'))
).astype(np.float32)  # better for this model


df = df.copy()

# select only those with recent bachelors drop the rest
keep = (df['DGRDG'] == 1) & (df['DGRYR'] >= 2021)
df = df.loc[keep].copy()

# drop the cols used to make y
df = df.drop(y_variables, axis=1).copy()

# drop the cols that cause memroy leaks, AI helped spot all of these
leak_vars = [
    # 1) Direct label vars
    "DGRDG","DGRYR","HDMN","STRTYR","STRTMN","WRKG","SALARY","OCEDRLP",
    "NRCHG","NRCON","NRFAM","NRLOC","NROCNA","NROT","NRPAY","NRREA","NRSEC",

    # 2) Job status / employment
    "HRSWK","WKSLYR","WKSWK","WKSYR","LFSTAT","LOOKWK","LWMN","LWYR","LWNVR",
    "NWFAM","NWILL","NWLAY","NWNOND","NWOCNA","NWOT","NWRET","NWRTYR","NWSTU",
    "PJFAM","PJHAJ","PJHRS","PJNOND","PJOCNA","PJOT","PJRET","PJRETYR","PJSTU",
    "FTPRET","FTPRTYR","WRKGP","SURV_SE","EDTP",

    # 3) Job satisfaction & benefits
    "JOBSATIS","SATADV","SATBEN","SATCHAL","SATIND","SATLOC","SATRESP","SATSAL","SATSEC","SATSOC",
    "JOBINS","JOBPENS","JOBPROFT","JOBVAC",

    # 4) Work activities
    "ACTCAP","ACTDED","ACTMGT","ACTRD","ACTRD2","ACTRDT","ACTRES","ACTTCH",
    "WAACC","WAAPRSH","WABRSH","WACOM","WADEV","WADSN","WAEMRL","WAMGMT","WAOT",
    "WAPRI","WAPROD","WAPRRD","WAPRSM","WAPRSM2","WAPRSM3","WAQM","WASALE",
    "WASCSM","WASCSM2","WASCSM3","WASVC","WATEA","WASEC",

    # 5) Employer & occupation
    "N2OCPRBG","N2OCPRMG","N3OCPR","N3OCPRNG","N3OCPRX",
    "N2OCBLST","N2OCMLST","N3OCLST","N3OCLSTX","N3OCNLST",
    "INDCODE","EMED","EMTP","EMSECDT","EMSECSM","EMSIZE","EMST_TOGA","EMUS",
    "EMRG","NEDTP","NEWBUS","PBPR21C","CARN21C","MGRNAT","MGROTH","MGRSOC",
    "SUPDIR","SUPIND","SUPWK","TELEC","TELEFR","PJWTFT","PRMBR","PROMTGI",

    # 6) Training & courses after degree
    "WKTRNI","WTRCHOC","WTREASN","WTREM","WTRLIC","WTROPPS","WTROT","WTRPERS","WTRSKL",
    "ACADV","ACCAR","ACCCEP","ACCHG","ACDRG","ACEM","ACFPT","ACGRD","ACINT",
    "ACLIC","ACOT","ACSIN","ACSKL","NACEDMG","NACEDNG",

    # 7) Survey design / admin
    "OBSNUM","SURID","SRVMODE","WTSURVY","COHORT","REFYR","BIRYR",
    # Optional: also drop TCDGCMP if present
    "TCDGCMP"
]

df = df.drop(columns=[c for c in leak_vars if c in df.columns])




# float32 mapping of objs, drop everything else that cant convert
yn_map = {'Y': 1, 'N': 0, 'y': 1, 'n': 0}
cols_to_drop = []

for col in df.columns:
    if df[col].dtype == 'object':
        s = df[col].replace(yn_map)
        converted = pd.to_numeric(s, errors='coerce') # object to NaN if failed
        # drop only if column is all NaN
        if converted.notna().sum() == 0:
            cols_to_drop.append(col)
        else:
            df[col] = converted

if cols_to_drop:
    df = df.drop(columns=cols_to_drop)

# cast the rest
num_cols = df.select_dtypes(include=['number']).columns
df[num_cols] = df[num_cols].astype('float32')

In [5]:
df.shape

(972, 337)

In [6]:
# check to see of all cols are float32
float32 = all(df.dtypes == 'float32')
print("All float32:", float32)

All float32: True


---
## Gradient Boosted Forest: Classification w/probability

In [7]:
X, y = df.drop(columns=['y']), df['y']

# train test splits
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    # random_state=67,
    test_size=0.2
)


# tuning parameters
params = {
    'n_estimators': [200, 300, 500],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 5],
    # 'gamma': [], # loss needed to partition further 
    'learning_rate': [0.01, 0.001, 0.05],
    'subsample': [0.8],
    'colsample_bytree': [0.66, 0.8], # fraction of cols to use on trees
    'lambda':[0.1, 1, 5], # regulate w on the conservitive side
    'alpha':[0, 0.1, 1] # same as lambda.. but diffrent
}

clas_model = xgbc(
    objective='binary:logistic', 
    n_jobs=1, # to make sure cores are used by sklearn
    tree_method='hist',
    eval_metric='logloss' # train accorting to log loss metric
)

clas_grid_search = GridSearchCV(
    estimator=clas_model,
    param_grid=params,
    cv=6,
    scoring='neg_log_loss',
    n_jobs=11,
    # verbose=2
)

# fit model
clas_grid_search.fit(X_train, y_train)
clas_best = clas_grid_search.best_estimator_

# evaluate the best model
probs = clas_best.predict_proba(X_test)[:, 1] # turns out this can do both!!
preds = clas_best.predict(X_test) # regular 0/1 labels

ll  = log_loss(y_test, probs)
auc = roc_auc_score(y_test, probs)
f1  = f1_score(y_test, preds)

print(
    '-'*30+'Model Performance Metrics'+'-'*30,
    f'\nLog Loss: {ll}',
    f'\nAUC: {auc}',
    f'\nF1 Score: {f1}',
)

------------------------------Model Performance Metrics------------------------------ 
Log Loss: 0.552834519702029 
AUC: 0.7921686746987953 
F1 Score: 0.7314285714285714


---
## Write scores and best parameters to a log file

In [8]:


logfile = "runs.txt"
with open(logfile, "a") as f:
    f.write('-'*30 + 'Model Performance Metrics' + '-'*30 + "\n")
    f.write(f"Timestamp: {datetime.now().isoformat(timespec='seconds')}\n")
    f.write(f"Log Loss: {ll}\n")
    f.write(f"AUC: {auc}\n")
    f.write(f"F1 Score: {f1}\n")
    f.write("Best Params:\n")
    for k, v in clas_grid_search.best_params_.items():
        f.write(f"  {k}: {v}\n")
    f.write("\n")

---
## Save best patamerts and features to a file

In [10]:
X_full = df.drop(columns=['y'])
y = df['y']

with open("best_params.json", "w") as f:
    json.dump(clas_grid_search.best_params_, f)

# save top features
top_k = 30
top_features = (
    pd.Series(clas_best.feature_importances_, index=X_full.columns)
      .sort_values(ascending=False)
      .head(top_k)
      .index
      .tolist()
)

with open("top_features.json", "w") as f:
    json.dump(top_features, f)

---
## Make a new model with only best features and perameters

In [56]:
# store clas_best features that resulted in the most loss in log loss on split
feat_imp = (
    pd.Series(clas_best.feature_importances_, index=X_full.columns)
      .sort_values(ascending=False)
)

# change to how ever many the model will use
top_k = 30
top_features = feat_imp.head(top_k).index.tolist()

print(top_features)

['GOVSUP', 'EARN', 'HDACY3', 'ND2MENG', 'CH6', 'HDGRD', 'MRGRD', 'CCST_TOGA', 'CH1218', 'UGFEM', 'CH25', 'HSYR', 'AGE', 'NBAMEBG', 'CHU2', 'CLICNOW', 'NMRMENG', 'FACSEC', 'FSHHS', 'UGFPLN', 'CLICEM', 'D2PBP21C', 'N2ACED', 'NDGMEMG', 'CCCOLPR', 'CHU2IN', 'N2ACEDX', 'NATIVE', 'CHUN12', 'N2D2MEDX']


In [57]:
X_small = X_full[top_features].copy()

X_train, X_test, y_train, y_test = train_test_split(
    X_small, y,
    test_size=0.2,
    random_state=67, # six seven
    stratify=y
)

best_params = clas_grid_search.best_params_
print(best_params)

{'alpha': 0, 'colsample_bytree': 0.66, 'lambda': 0.1, 'learning_rate': 0.01, 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 500, 'subsample': 0.8}


In [58]:
reduced_model = xgbc(
    objective='binary:logistic',
    n_jobs=1,
    tree_method='hist',
    eval_metric='logloss',
    random_state=67, # six seven
    **best_params # unpack
)

reduced_model.fit(X_train, y_train)

probs_small = reduced_model.predict_proba(X_test)[:, 1]
preds_small = reduced_model.predict(X_test)

print(
    "-"*30 + " Reduced Model " + "-"*30,
    f"\nLog Loss: {log_loss(y_test, probs_small)}",
    f"\nAUC: {roc_auc_score(y_test, probs_small)}",
    f"\nF1 Score: {f1_score(y_test, preds_small)}",
)

------------------------------ Reduced Model ------------------------------ 
Log Loss: 0.4867325938664912 
AUC: 0.8514186701321204 
F1 Score: 0.6709677419354839
